[![Binder](img/badge-binder.svg)](https://mybinder.org/v2/gh/nhirschey/teaching/gh-pages?filepath=assignment-performance-evaluation.ipynb)&emsp;
[![Script](img/badge-script.svg)](/Teaching//assignment-performance-evaluation.fsx)&emsp;
[![Notebook](img/badge-notebook.svg)](/Teaching//assignment-performance-evaluation.ipynb)

Student Name | Student Number

**1** | Nicholas Sistovaris | 49606

**2** | Omar ben Ayed | 38628


**Signal Name (e.g., Book to Market):** Change in Quarterly Return on Assets

**Signal Code (e.g., be_me):** niq_at_chg1  

This is an assignment. You may work in pairs (two students) using either student's signal to answer the below questions.  You will find sections labeled **Task** asking you to do each piece of analysis. Please make sure that you complete all of these tasks. Make use of the course resources and example code on the course website. It should be possible to complete all the requested tasks using information given below or somewhere on the course website.


In [ ]:
#r "nuget:FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET, 2.0.0-preview.17"

Installed Packages FSharp.Data, 4.2.8 FSharp.Stats, 0.4.5 Plotly.NET, 2.0.0-preview.17

In [ ]:
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.17"


Installed Packages Plotly.NET.Interactive, 2.0.0-preview.17

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
open System
open FSharp.Data
open Plotly.NET
open FSharp.Stats

In [ ]:
// Set dotnet interactive formatter to plaintext
Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x )
Formatter.SetPreferredMimeTypesFor(typeof<obj>, "text/plain")
// Make plotly graphs work with interactive plaintext formatter
Formatter.SetPreferredMimeTypesFor(typeof<GenericChart.GenericChart>,"text/html")


## Load Data

First, make sure that you're referencing the correct files.

Here I'm assuming that you have a class folder with this
notebook and these files in it. The folder hierarchy would
look like below where you have the below files and folders accessible.

* `Common.fsx` is on the course website.

* `notebook.ipynb` is this notebook.

* `myExcessReturnPortfolios.csv` is generated by code for construction [signal portfolios](signal-portfolio.html).

```code
/class
    Common.fsx
    notebook.ipynb                
    /data
        myExcessReturnPortfolios.csv
```


In [ ]:
let [<Literal>] ResolutionFolder = __SOURCE_DIRECTORY__
Environment.CurrentDirectory <- ResolutionFolder

#load "Common.fsx"
open Common


We get the Fama-French 3-Factor asset pricing model data.



In [ ]:
let ff3 = French.getFF3 Frequency.Monthly

### Data file



In [ ]:
let [<Literal>] myExcessReturnPortfoliosPath = "data/myExcessReturnPortfolios.csv"

If my paths are correct, then this code should read the first few lines of the files.
If it doesn't show the first few lines, fix the above file paths.



In [ ]:
IO.File.ReadLines(myExcessReturnPortfoliosPath) |> Seq.truncate 5

seq
  ["portfolioName(string),index(int option),yearMonth(date),ret(float)";
   "Change in quarterly return on assets,1,2000-02-01T00:00:00.0000000,0.034519524";
   "Change in quarterly return on assets,2,2000-02-01T00:00:00.0000000,-0.052082942";
   "Change in quarterly return on assets,3,2000-02-01T00:00:00.0000000,0.084206766"; ...]


Ok, now assuming those paths were correct the below code will work.
I will put all this prep code in one block so that it is easy to run.



In [ ]:
let myExcessReturnPortfolios = 
    CsvProvider<myExcessReturnPortfoliosPath,
                ResolutionFolder = ResolutionFolder>.GetSample()

Headers



In [ ]:
myExcessReturnPortfolios.Headers

Some [|"portfolioName(string)"; "index(int option)"; "yearMonth(date)"; "ret(float)"|]


First few rows



In [ ]:
myExcessReturnPortfolios.Rows
|> Seq.take 3

seq
  [("Change in quarterly return on assets", Some 1, 2/1/2000 12:00:00 AM, 0.034519524);
   ("Change in quarterly return on assets", Some 2, 2/1/2000 12:00:00 AM, -0.052082942);
   ("Change in quarterly return on assets", Some 3, 2/1/2000 12:00:00 AM, 0.084206766)]


Isolate some notable portfolios.



In [ ]:
type SignalPortfolioObs = 
    { Month: DateTime
      Name: string
      Return: float }
let long =
    myExcessReturnPortfolios.Rows
    |> Seq.toList
    |> List.filter (fun row -> row.Index = Some 3)
    |> List.map (fun row -> 
        { Month = row.YearMonth
          Name = "Long"
          Return = row.Ret })

let short =
    myExcessReturnPortfolios.Rows
    |> Seq.toList
    |> List.filter (fun row -> row.Index = Some 1)
    |> List.map (fun row -> 
        { Month = row.YearMonth
          Name = "Short"
          Return = row.Ret })

let shortByMonth = 
    short 
    |> List.map (fun x -> x.Month, x) 
    |> Map

let longShort =
    [ for l in long do 
        // I'm indexing into shortByMonth directly
        // rather than using Map.tryFind because
        // I am pretty confident that the short 
        // data will be there
        let s = shortByMonth[l.Month]
        { Month = l.Month
          Name = "Long-short"
          Return = l.Return - s.Return }]


## Start of assignment

> **Task:** Calculate the annualized Sharpe ratios of your long, short, and long-short portfolios.
> 



In [ ]:
long.GetType()

Microsoft.FSharp.Collections.FSharpList`1[FSI_0017+SignalPortfolioObs]


In [ ]:
// Solution here.
/// Calculates sharpe ratio of a sequence of excess returns
let sharpe (xs: float seq) = (Seq.mean xs) / (Seq.stDev xs)

let annualizeMonthlySharpe monthlySharpe = sqrt(12.0) * monthlySharpe

In [ ]:
let long_sharpe = 
long
|> List.map (fun x -> x.Return)
|> sharpe
|> annualizeMonthlySharpe

In [ ]:
long_sharpe

0.422867039


In [ ]:
let short_sharpe = 
short
|> List.map (fun x -> x.Return)
|> sharpe
|> annualizeMonthlySharpe

In [ ]:
short_sharpe

0.3199988029


In [ ]:
let longShort_sharpe = 
longShort
|> List.map (fun x -> x.Return)
|> sharpe
|> annualizeMonthlySharpe

In [ ]:
longShort_sharpe

0.1700084013


> **Task:** Estimate CAPM and Fama-French 3-factor models for your long-short portfolio.
> 



In [ ]:
// Solution here.
#r "nuget: Accord"
#r "nuget: Accord.Statistics"

open Accord
open Accord.Statistics.Models.Regression.Linear

Microsoft.DotNet.Interactive.InstallPackagesMessage


In [ ]:
type RegressionOutput =
    { Model : MultipleLinearRegression 
      Intercept: float 
      TValuesWeights : float array
      TValuesIntercept : float 
      R2: float }

/// Type alias for x, y regression data 
type XY = (float array) array * float array

let fitModel (x: (float array) array, y: float array) =
    let ols = new OrdinaryLeastSquares(UseIntercept=true)
    let estimate = ols.Learn(x,y)
    let mse = estimate.GetStandardError(x,y)
    let se = estimate.GetStandardErrors(mse, ols.GetInformationMatrix())
    let tvaluesWeights = 
        estimate.Weights
        |> Array.mapi(fun i w -> w / se.[i])
    let tvalueIntercept = estimate.Intercept / (se |> Array.last)
    let r2 = estimate.CoefficientOfDetermination(x,y)
    { Model = estimate
      Intercept = estimate.Intercept
      TValuesWeights = tvaluesWeights
      TValuesIntercept = tvalueIntercept  
      R2 = r2 }

In [ ]:
longShort |> List.truncate 2

[{ Month = 2/1/2000 12:00:00 AM
   Name = "Long-short"
   Return = 0.049687242 }; { Month = 3/1/2000 12:00:00 AM
                             Name = "Long-short"
                             Return = -0.049338013 }]


In [ ]:
ff3 |> Array.truncate 2

[|{ Date = 7/1/1926 12:00:00 AM
    MktRf = 0.0296
    Smb = -0.0238
    Hml = -0.0273
    Rf = 0.0022
    Frequency = Monthly }; { Date = 8/1/1926 12:00:00 AM
                             MktRf = 0.0264
                             Smb = -0.0147
                             Hml = 0.0414
                             Rf = 0.0025
                             Frequency = Monthly }|]


In [ ]:
let ff3ByMonth = 
    ff3
    |> Array.map(fun x -> DateTime(x.Date.Year, x.Date.Month,1), x)
    |> Map

In [ ]:
type RegData =
    { Date : DateTime
      Name : string
      Return : float
      Portfolio : float
      MktRf : float 
      Hml : float 
      Smb : float }

In [ ]:
let regData =
    longShort 
    |> List.map(fun port ->
        let monthToFind = DateTime(port.Month.Year,port.Month.Month,1)
        match Map.tryFind monthToFind ff3ByMonth with
        | None -> failwith "probably you messed up your days of months"
        | Some ff3 -> 
            { Date = monthToFind
              Name = port.Name
              Return = port.Return
              Portfolio = port.Return - ff3.Rf
              MktRf = ff3.MktRf 
              Hml = ff3.Hml 
              Smb = ff3.Smb })

In [ ]:
let capmModelData = 
    regData
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip 

In [ ]:
let ff3ModelData = 
    regData
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf; obs.Hml; obs.Smb |], obs.Portfolio)
    |> Array.unzip

In [ ]:
let capmEstimate = capmModelData |> fitModel

In [ ]:
capmEstimate

{ Model = y(x0) = -0.09457504542673835*x0 + 0.0002652518681141018
  Intercept = 0.0002652518681
  TValuesWeights = [|-3.382760895|]
  TValuesIntercept = 0.2065324055
  R2 = 0.04393693875 }


In [ ]:
let ff3Estimate = ff3ModelData |> fitModel

In [ ]:
ff3Estimate

{ Model =
   y(x0, x1, x2) = -0.11796522002267656*x0 + -0.03747800861527988*x1 + 0.11230487596158913*x2 + 0.00017037481892364246
  Intercept = 0.0001703748189
  TValuesWeights = [|-4.085636848; -0.9542134769; 2.707969195|]
  TValuesIntercept = 0.1345202644
  R2 = 0.07979224469 }


In [ ]:
type Prediction = { Label : float; Score : float}

let makePredictions 
    (estimate:MultipleLinearRegression) 
    (x: (float array) array, y: float array) =
    (estimate.Transform(x), y)
    ||> Array.zip
    |> Array.map(fun (score, label) -> { Score = score; Label = label })

let residuals (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)

let capmPredictions = makePredictions capmEstimate.Model capmModelData
let ff3Predictions = makePredictions ff3Estimate.Model ff3ModelData

capmPredictions |> Array.take 3

[|{ Label = 0.045387242
    Score = -0.002051836745 }; { Label = -0.054038013
                                 Score = -0.004652650494 }; { Label = 0.060388454
                                                              Score = 0.006318054775 }|]


### CAPM model evaluation.

> **Task:** What is the CAPM alpha for your long-short portfolio. Is it significantly different from zero?
> 



In [ ]:
// not sure whether to use monthly or annualized
let capmAlpha = 12.0 * capmEstimate.Model.Intercept 

capmAlpha

0.003183022417


Our alpha is equal to 0.3 %, this is not a lot unless one can introduce leverage to the strategy. Since our T-value is close to 0 (smaller than 1.645) we cannot state that our alpha is significant.

In [ ]:
// Solution here.
let capm_alpha = capmEstimate.Intercept

capm_alpha

0.0002652518681


> **Task:** What is the CAPM beta for your long-short portfolio. Is it significantly different from zero?
> 



In [ ]:
capmEstimate.Model.Weights[0]

-0.09457504543


**Description**

Annualized the beta for the long-short portfolio is significantly different from 0. Annualized it currently stands at **-5.72** which would suggest that the it will reduce the risk of the portofolio significantly.

In [ ]:
// Solution here.
let capmBeta = 12.0 * capmEstimate.Model.Weights[0]

capmBeta

-1.134900545


> **Task:** What is the information ratio for your long-short portfolio when using the CAPM model?
> 



In [ ]:
let capmResiduals = residuals capmPredictions
let ff3Residuals = residuals ff3Predictions

In [ ]:
// Function version

let informationRatio monthlyAlpha (monthlyResiduals: float array) =
    let annualAlpha = 12.0 * monthlyAlpha
    let annualStDev = sqrt(12.0) * (Seq.stDev monthlyResiduals)
    annualAlpha / annualStDev 

informationRatio capmEstimate.Model.Intercept capmResiduals


0.04560071271


### Fama-French 3-factor model evaluation.

> **Task:** What is the Fama-French 3-factor model alpha for your long-short portfolio. Is it significantly different from zero?
> 



In [ ]:
// not sure whether to use monthly or annualized
let ff3Alpha = 12.0 * ff3Estimate.Model.Intercept 

ff3Alpha

0.002044497827


Our alpha is equal to 0.2 %, this is not a lot unless one can introduce leverage to the strategy. Since our T-value is close to 0 (smaller than 1.645) we cannot state that our alpha is significant.

> **Task:** What are the betas on the Market, HML, and SMB factors for your long-short portfolio. Are they significantly different from zero?
> 



In [ ]:
// Solution here.
let ff3Market = ff3Estimate.Model.Weights[0]

ff3Market

-0.11796522


In [ ]:
// Solution here.
let ff3HML = ff3Estimate.Model.Weights[1]

ff3HML

-0.03747800862


In [ ]:
// Solution here.
let ff3SMB = ff3Estimate.Model.Weights[2]

ff3SMB

0.112304876


> **Task:** Based on the Market, HML, and SMB factor betas for your long-short portfolio, would you say your portfolio is more like a value portfolio, more like a growth portfolio, or neither?
> 



Our portfolio has a HML beta of -0.037. The portfolio would then theoretically lean more into the growth category. However, since the Beta is very close to 0: we could say that the portfolio seeks risk neutrality when it comes to Book-to-Market values.

> **Task:** Based on the Market, HML, and SMB factor betas for your long-short portfolio, would you say your portfolio is more like a small-cap portfolio, more like a large-cap portfolio, or neither?
> 



Our portfolio has a SMB beta of 0.11. The portofolio seems to be small stock biased.

> **Task:** What is the information ratio for your long-short portfolio when using the Fama and French 3-factor model?
> 



In [ ]:
let ff3SigmaResiduals = (Seq.stDev ff3Residuals) * sqrt(12.0)
let ff3InformationRatio = ff3Alpha / ff3SigmaResiduals

ff3InformationRatio


0.02985512621


The information Ratio of our long-short portfolio is equal to around 0.03.